Start with defining start time

In [ ]:
import os


def create_dir(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
        
from multiply_data_access import DataAccessComponent
from multiply_orchestration import create_sym_links

def get_static_data(data_access_component: DataAccessComponent, roi: str, start_time: str, stop_time: str, emulation_directory: str, dem_directory: str):
    create_dir(emulation_directory)
    create_dir(dem_directory)
    emu_urls = data_access_component.get_data_urls(roi, start_time, stop_time, 'ISO_MSI_A_EMU,ISO_B_EMU')
    create_sym_links(emu_urls, emulation_directory)
    dem_urls = data_access_component.get_data_urls(roi, start_time, stop_time, 'Aster DEM')
    create_sym_links(dem_urls, dem_directory)


import datetime


def get_dynamic_data(data_access_component: DataAccessComponent, roi: str, start_time: str, stop_time: str, modis_directory: str, cams_tiff_directory: str,
                     s2_l1c_directory: str):
    create_dir(modis_directory)
    create_dir(cams_tiff_directory)
    create_dir(s2_l1c_directory)
    
    modis_delta = datetime.timedelta(days=16)
    start = datetime.datetime.strptime(start_time, '%Y-%m-%d')
    modis_start = start - modis_delta
    modis_start_time = datetime.datetime.strftime(modis_start, '%Y-%m-%d')
    end = datetime.datetime.strptime(stop_time, '%Y-%m-%d')
    modis_end = end + modis_delta
    modis_end_time = datetime.datetime.strftime(modis_end, '%Y-%m-%d')
    
    modis_urls = data_access_component.get_data_urls(roi, modis_start_time, modis_end_time, 'MCD43A1.006')
    create_sym_links(modis_urls, modis_directory)
    cams_urls = data_access_component.get_data_urls(roi, start_time, stop_time, 'CAMS_TIFF')
    create_sym_links(cams_urls, cams_tiff_directory)
    s2_urls = data_access_component.get_data_urls(roi, start_time, stop_time, 'AWS_S2_L1C')
    create_sym_links(s2_urls, s2_l1c_directory)


import glob


def preprocess(s2_l1c_dir: str, modis_dir: str, emus_dir: str, cams_dir: str, dem_dir: str, output_dir):
    vrt_dem_file = glob.glob(dem_dir+'/'+'*.vrt')[0]
    processor_dir = '/software/atmospheric_correction_2.0.9/SIAC'
    create_dir(output_dir)
    dirs = glob.glob(s2_l1c_dir + "/*/*/*/*/*/*/*")
    for dir in dirs:
        input_dir = dir[len(s2_l1c_dir)+1:]
        input_parts = input_dir.split('/')
        product_name = "S2-" + input_parts[-7] + input_parts[-6] + input_parts[-5]  + input_parts[-4] + "-" +  input_parts[-3] + "-" +  input_parts[-2]
        output_dir = output_root_dir+'/'+product_name+'/' 
        os.system("PYTHONPATH=$PYTHONPATH:"+processor_dir+"/util python "+processor_dir+"/SIAC_S2.py -f "+s2_l1c_dir+'/'+input_dir+"/ -m "+ 
                  modis_dir + " -e "+emus_dir+" -c "+cams_dir + " -d "+vrt_dem_file+" -o False")
        create_dir(output_dir)
        os.system("mv $(find "+s2_l1c_dir+'/'+input_dir+'/ -type f) '+output_dir+'/')
        os.system("cp `readlink "+s2_l1c_dir+'/'+input_dir+"/metadata.xml` "+output_dir+"/metadata.xml")


def put_data(data_access_component: DataAccessComponent, sdrs_directory: str):
    for sdr in os.listdir(sdrs_directory):
	    data_access_component.put(sdrs_dir+'/'+sdr, 'S2L2')

        
from typing import List
from vm_support import create_config_file


def get_priors(temp_dir: str, roi: str, start_time: str, end_time: str, time_interval: str, priors_directory: str, parameter_list: List[str]):
    config_file = create_config_file(temp_dir, roi, start_time, end_time, time_interval, priors_directory, parameter_list)
    time = datetime.datetime.strptime(start, '%Y-%m-%d')
    end_time = datetime.datetime.strptime(end, '%Y-%m-%d')
    create_dir(priors_directory)
    while time<=end_time:
        prior_engine = PriorEngine(config=config_file, datestr=time.strftime('%Y-%m-%d'), variables=variables)
        prior_engine.get_priors()
        time = time + datetime.timedelta(days=1)
        

def infer(roi: str, start_date: str, end_date: str, previous_state: str, priors_directory: str, sdrs_directory: str, 
          next_state: str, biophys_dir: str, parameter_list: List[str], state_mask: Optional[str], 
          spatial_resolution: Optional[int], roi_grid: Optional[str], destination_grid: Optional[str]):
    processor_dir = '/software/inference-engine-0.4/multiply_inference_engine'
    s2_emulators_dir = '/data/archive/emulators/s2_prosail'
    inference_type = 'high'
    create_dir(next_state)
    create_dir(biophys_dir)
    os.system("python "+processor_dir+"/inference_engine.py"
          + " -s " + start_date
          + " -e " + end_date
          + " -i " + inference_type
          + " -em " + s2_emulators_dir
          + " -p " + parameter_list
          + ((" -ps " + previous_state) if previous_state != 'none' else "")
          + " -pd " + priors_directory
          + " -d " + sdrs_directory
          + " -sm " + state_mask
          + " -o " + biophys_dir
          + " -roi \'" + roi + "\'"
          + " -res " + str(spatial_resolution)
          + " -rg " + roi_grid
          + " -dg " + destination_grid
          + " -ns " + next_state)

In [ ]:
from vm_support import set_earth_data_authentication

set_earth_data_authentication(username, password) # to download modis data, needs only be done once

roi = 'POLYGON((-2.20397502663252 39.09868106889479,-1.9142106223355313 39.09868106889479,-1.9142106223355313 38.94504502508093,' \
      '-2.20397502663252 38.94504502508093,-2.20397502663252 39.09868106889479))'
roi_grid = 'EPSG:4362'
spatial_resolution = 60 # in m
destination_grid = 'EPSG:32632'

from vm_support import get_working_dir

working_dir = get_working_dir('m1')

priors_directory = '{}/priors'.format(working_dir)
modis_directory = '{}/modis'.format(working_dir)
state_directory = '{}/state'.format(working_dir)
cams_directory = '{}/cams'.format(working_dir)
s2_l1c_directory = '{}/s2'.format(working_dir)
sdrs_directory = '{}/priors'.format(working_dir)
biophys_output = '{}/biophys'.format(working_dir)
emulators_directory = '{}/emulators'.format(working_dir)
dem_directory = '{}/dem'.format(working_dir)

start_time_as_string = '2017-06-01'
stop_time_as_string = '2017-06-30'
start_time_as_datetime = datetime.datetime.strptime(start_time_as_string, '%Y-%m-%d')
stop_time_as_datetime = datetime.datetime.strptime(stop_time_as_string, '%Y-%m-%d')
time_step = 5 # in days
time_step_as_time_delta = datetime.timedelta(days=time_step)

parameters = {'n', 'cab', 'car', 'cb', 'cw', 'cdm', 'lai', 'ala', 'bsoil', 'psoil'}

data_access_component = DataAccessComponent()

get_static_data(data_access_component=data_access_component, roi=roi, start_time=start_time_as_string, stop_time=stop_time_as_string, 
                emulation_directory=emulators_directory, dem_directory=dem_directory)

cursor = start_time_as_datetime
previous_inference_state = 'none'
updated_inference_state = 'none'
one_day_step = datetime.timedelta(days=1)

while cursor <= stop_time_as_datetime:
    date_as_string = datetime.datetime.strftime(cursor, '%Y-%m-%d')
    cursor += time_step_as_time_delta
    cursor -= one_day_step
    if cursor > stop_time_as_datetime:
        cursor = stop_time_as_datetime
    next_date_as_string = datetime.datetime.strftime(cursor, '%Y-%m-%d')
    cursor += one_day_step
    
    modis_directory_for_date = '{}/{}'.format(modis_directory, date_as_string)
    cams_directory_for_date = '{}/{}'.format(cams_directory, date_as_string)
    s2_l1c_directory_for_date = '{}/{}'.format(s2_l1c_directory, date_as_string)
    sdrs_directory_for_date = '{}/{}'.format(sdrs_directory, date_as_string)
    get_dynamic_data(data_access_component, roi, date_as_string, next_date_as_string, 
                     modis_directory_for_date, cams_directory_for_date, s2_l1c_directory_for_date)
    
    preprocess(s2_l1c_directory_for_date, modis_directory_for_date, emulators_directory, cams_directory_for_date, dem_directory, sdrs_directory_for_date)
    
    put_data(data_access_component, sdrs_directory_for_date)
    
    priors_directory_for_date = '{}/{}'.format(priors_directory, date_as_string)
    
    get_priors(working_dir, roi, date_as_string, next_date_as_string, time_step, priors_directory_for_date, parameters)
    
    updated_inference_state = '{}/{}'.format(previous_inference_state, date_as_string)
    infer(roi, date_as_string, next_date_as_string, previous_inference_state, priors_directory_for_date, sdrs_directory_for_date, updated_inference_state,
          biophys_output, parameters, None, spatial_resolution, roi_grid, destination_grid)
    previous_inference_state = updated_inference_state